In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os 
import numpy as np
import pandas as pd 
from sklearn import preprocessing
from sklearn.impute import SimpleImputer 
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier

In [3]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Bert/data.csv"
df = pd.read_csv(path) 
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,Level 5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,Level 5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,Level 5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",Level 5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Level 2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [4]:
df.iloc[:,1].replace(['Level 1', 'Level 2', 'Level 3', 'Level 4', 'Level 5'],
                        [1, 2, 3, 4, 5], inplace=True)

In [5]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,5,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,5,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,5,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",5,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,2,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [6]:
#define bins
bins = [0,2,4,5]
#name of the group 
group_names = ['Low', 'Moderate', 'High']
df['Level '] = pd.cut(df["Level "],bins,labels = group_names)

In [7]:
df.head(5)

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,High,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,High,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,High,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",High,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,Low,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [8]:
df['Level '] = df['Level '].map({'Low': 1, 'Moderate': 2, 'High': 3}).astype(int)

In [9]:
df.head()

,Problem,Level,Type,Solution,Extracted words(List),Extracted Num(List)
0,There are constants alpha and beta suc...,3,Algebra,The numerator x^2 - 80x + 1551 factors as (...,"['There', 'are', 'constants', 'alpha', 'and', ...","['2', '80', '1551', '2', '57', '2970']"
1,The length of the segment between the points ...,3,Algebra,"By the distance formula, the distance from (2...","['The', 'length', 'of', 'the', 'segment', 'bet...","['2', '4', '4', '1', '2', '10']"
2,Chewbacca has 20 pieces of cherry gum and 30 p...,3,Algebra,"If Chewbacca loses one pack of cherry gum, the...","['Chewbacca', 'has', 'pieces', 'of', 'cherry',...","['20', '30', '5']"
3,"For some constants a and b, let [f(x) = ...",3,Algebra,"Setting x = 3, we get f(3) = -1. Since -1...","['For', 'some', 'constants', 'a', 'and', 'b', ...","['2', '8', '3', '2']"
4,Calculate the sum 1 + 3 + 5 + cdots + 15 + ...,1,Algebra,"The arithmetic sequence 1, 3, 5, dots , 17,...","['Calculate', 'the', 'sum', 'cdots']","['1', '3', '5', '15', '17']"


In [10]:
import pandas as pd
path = "/content/drive/MyDrive/newpro/threeclass_problem/ANOVA_Bert/bert_withstopwordchangelevels_200_features.csv"
x = pd.read_csv(path)
x.head() 

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,190,191,192,193,194,195,196,197,198,199
0,0,0.477472,1.284731,-1.490585,-0.448166,-0.698320,0.272613,-0.549095,-1.836217,-0.041156,...,1.056721,0.265053,-0.739267,1.863160,-0.209238,-0.853295,0.981719,-1.247515,-1.111566,0.249373
1,1,-0.784304,-0.484810,0.615874,0.752646,0.704698,0.074672,0.688078,0.704383,-0.629409,...,0.065721,-1.002811,0.408338,-0.501279,-0.329111,0.264686,-0.761664,1.298238,1.106121,-0.348662
2,2,-0.044236,0.219938,0.036338,-0.246541,-0.658413,0.298457,-0.447984,0.419400,0.793930,...,0.908678,0.442698,-0.400353,0.758424,-0.072104,-0.513999,0.414647,-0.250666,-0.637803,0.499352
3,3,1.560440,1.715057,-1.183646,-1.172434,-0.996845,-0.377401,-0.766910,-1.852115,1.742404,...,2.542506,0.306135,-1.502726,1.303629,1.291529,-1.925317,1.513284,-0.030323,-1.434217,1.855407
4,4,-0.718876,-1.484737,-0.207207,1.152390,1.357838,1.032013,0.446249,0.765055,-0.749607,...,-1.022358,-0.512520,0.666394,-0.176417,-0.820211,0.632367,-1.304105,1.585201,1.094319,-0.689731


In [11]:
x.drop(x.columns[[0]], axis=1, inplace=True)
x.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.477472,1.284731,-1.490585,-0.448166,-0.698320,0.272613,-0.549095,-1.836217,-0.041156,-1.056629,...,1.056721,0.265053,-0.739267,1.863160,-0.209238,-0.853295,0.981719,-1.247515,-1.111566,0.249373
1,-0.784304,-0.484810,0.615874,0.752646,0.704698,0.074672,0.688078,0.704383,-0.629409,0.019055,...,0.065721,-1.002811,0.408338,-0.501279,-0.329111,0.264686,-0.761664,1.298238,1.106121,-0.348662
2,-0.044236,0.219938,0.036338,-0.246541,-0.658413,0.298457,-0.447984,0.419400,0.793930,-0.627094,...,0.908678,0.442698,-0.400353,0.758424,-0.072104,-0.513999,0.414647,-0.250666,-0.637803,0.499352
3,1.560440,1.715057,-1.183646,-1.172434,-0.996845,-0.377401,-0.766910,-1.852115,1.742404,-0.790168,...,2.542506,0.306135,-1.502726,1.303629,1.291529,-1.925317,1.513284,-0.030323,-1.434217,1.855407
4,-0.718876,-1.484737,-0.207207,1.152390,1.357838,1.032013,0.446249,0.765055,-0.749607,0.532769,...,-1.022358,-0.512520,0.666394,-0.176417,-0.820211,0.632367,-1.304105,1.585201,1.094319,-0.689731


In [12]:
y=df['Level '] 

**Train Test Split**

In [13]:
# Train-Test-Split is performed

from sklearn.model_selection import train_test_split

# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=42)
X_train.shape, X_test.shape

((1220, 200), (524, 200))

In [14]:
print(y_train.value_counts(normalize=True)*100)
print("\n")
print(y_test.value_counts(normalize=True)*100)

2    45.327869
1    29.590164
3    25.081967
Name: Level , dtype: float64


2    45.229008
1    29.961832
3    24.809160
Name: Level , dtype: float64


**Model Building**

In [ ]:
def perceptron_classifier (X_train, y_train, X_test, y_test):
  p = Perceptron(random_state=150)
  p.fit(X_train, y_train)
  predictions_train = p.predict(X_train)
  predictions_test = p.predict(X_test)
  train_score = accuracy_score(predictions_train, y_train)
  #print("score on train data: ", train_score)
  test_score = f1_score(predictions_test, y_test, average='weighted' )
  print("Perceptron F1 score:", test_score)

In [ ]:
from sklearn.linear_model import Perceptron
perceptron_classifier(X_train, y_train, X_test, y_test) 

Perceptron F1 score: 0.41131968192939483


**Decision Tree**

In [15]:
clf_dt = DecisionTreeClassifier(max_depth=5, class_weight = "balanced")

In [16]:
clf_dt.fit(X_train,y_train)

DecisionTreeClassifier(class_weight='balanced', max_depth=5)

In [17]:
y_pred_train = clf_dt.predict(X_train)
y_pred_test = clf_dt.predict(X_test)

In [18]:
def classifcation_report_train_test(y_train, y_train_pred, y_test, y_test_pred):

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TRAIN DATA
            =========================================
            ''')
    print(classification_report(y_train, y_train_pred, digits=4))

    print('''
            =========================================
               CLASSIFICATION REPORT FOR TEST DATA
            =========================================
            ''')
    print(classification_report(y_test, y_test_pred, digits=4))

In [19]:
scores = pd.DataFrame(columns=['Model', 'Train_Accuracy', 'Train_Recall', 'Train_Precision', 'Train_F1_Score', 
                               'Test_Accuracy', 'Test_Recall', 'Test_Precision', 'Test_F1_Score'])

def get_metrics(train_actual, train_predicted, test_actual, test_predicted, model_description, dataframe):

    train_accuracy  = accuracy_score(train_actual, train_predicted)
    train_recall    = recall_score(train_actual, train_predicted, average="weighted")
    train_precision = precision_score(train_actual, train_predicted, average="weighted")
    train_f1score   = f1_score(train_actual, train_predicted, average="weighted")
    
    test_accuracy   = accuracy_score(test_actual, test_predicted)
    test_recall     = recall_score(test_actual, test_predicted, average="weighted")
    test_precision  = precision_score(test_actual, test_predicted, average="weighted")
    test_f1score    = f1_score(test_actual, test_predicted, average="weighted")

    dataframe       = dataframe.append(pd.Series([model_description, 
                                                  train_accuracy, train_recall, train_precision, train_f1score,
                                                  test_accuracy, test_recall, test_precision, test_f1score],
                                                 index=scores.columns ), 
                                       ignore_index=True)

    return(dataframe)

In [20]:
scores = get_metrics(y_train, y_pred_train, y_test, y_pred_test, "DecisionTrees", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.480328,0.480328,0.512694,0.482128,0.332061,0.332061,0.355855,0.333659


**Random Forest**

In [21]:
clf_rf = RandomForestClassifier(class_weight = "balanced")

In [22]:
clf_rf.fit(X=X_train, y=y_train)

RandomForestClassifier(class_weight='balanced')

In [23]:
y_pred_train1 = clf_rf.predict(X_train)
y_pred_test1 = clf_rf.predict(X_test)

In [24]:
classifcation_report_train_test(y_train, y_pred_train1, y_test, y_pred_test1)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     1.0000    1.0000    1.0000       361
           2     1.0000    1.0000    1.0000       553
           3     1.0000    1.0000    1.0000       306

    accuracy                         1.0000      1220
   macro avg     1.0000    1.0000    1.0000      1220
weighted avg     1.0000    1.0000    1.0000      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2604    0.1592    0.1976       157
           2     0.4584    0.7679    0.5741       237
           3     0.1935    0.0462    0.0745       130

    accuracy                         0.4065       524
   macro avg     0.3041    0.3244    0.2821       524
weighted avg     0.3334    0.4065    0.3374       524



In [25]:
scores = get_metrics(y_train, y_pred_train1, y_test, y_pred_test1, "RandomForest", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.480328,0.480328,0.512694,0.482128,0.332061,0.332061,0.355855,0.333659
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.406489,0.406489,0.333390,0.337379


**AdaBoost**

In [ ]:
clf_adaboost =  AdaBoostClassifier()

In [ ]:
clf_adaboost.fit(X_train, y_train)

AdaBoostClassifier()

In [ ]:
y_pred_train2 = clf_adaboost.predict(X_train)
y_pred_test2 = clf_adaboost.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train2, y_test, y_pred_test2)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5401    0.3546    0.4281       361
           2     0.5633    0.8047    0.6627       553
           3     0.5544    0.3497    0.4289       306

    accuracy                         0.5574      1220
   macro avg     0.5526    0.5030    0.5065      1220
weighted avg     0.5542    0.5574    0.5346      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3577    0.2803    0.3143       157
           2     0.4854    0.7004    0.5734       237
           3     0.2542    0.1154    0.1587       130

    accuracy                         0.4294       524
   macro avg     0.3658    0.3654    0.3488       524
weighted avg     0.3898    0.4294    0.3929       524



In [ ]:
scores = get_metrics(y_train, y_pred_train2, y_test, y_pred_test2, "Adaboost", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.543443,0.543443,0.586143,0.484771,0.427481,0.427481,0.389838,0.362217
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.442748,0.442748,0.388654,0.382806
2,Adaboost,0.557377,0.557377,0.554195,0.534625,0.429389,0.429389,0.389787,0.392890


**Gradient Boosting Classifier**

In [ ]:
clf_gbm = GradientBoostingClassifier()

In [ ]:
clf_gbm.fit(X=X_train, y=y_train)

GradientBoostingClassifier()

In [ ]:
y_pred_train3 = clf_gbm.predict(X_train)
y_pred_test3 = clf_gbm.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train3, y_test, y_pred_test3)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.9841    0.8560    0.9156       361
           2     0.8565    0.9928    0.9196       553
           3     0.9849    0.8529    0.9142       306

    accuracy                         0.9172      1220
   macro avg     0.9418    0.9006    0.9164      1220
weighted avg     0.9264    0.9172    0.9170      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3579    0.2166    0.2698       157
           2     0.4866    0.7637    0.5944       237
           3     0.2281    0.1000    0.1390       130

    accuracy                         0.4351       524
   macro avg     0.3575    0.3601    0.3344       524
weighted avg     0.3839    0.4351    0.3842       524



In [ ]:
scores = get_metrics(y_train, y_pred_train3, y_test, y_pred_test3, "GBM", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.543443,0.543443,0.586143,0.484771,0.427481,0.427481,0.389838,0.362217
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.442748,0.442748,0.388654,0.382806
2,Adaboost,0.557377,0.557377,0.554195,0.534625,0.429389,0.429389,0.389787,0.392890
3,GBM,0.917213,0.917213,0.926445,0.917044,0.435115,0.435115,0.383880,0.384192


**SVM Model**

In [26]:
linear_svm = SVC(kernel='linear', C=1, class_weight = "balanced")

In [27]:
linear_svm.fit(X=X_train, y= y_train)

SVC(C=1, class_weight='balanced', kernel='linear')

In [28]:
y_pred_train4 = linear_svm.predict(X_train)
y_pred_test4 = linear_svm.predict(X_test)

In [29]:
classifcation_report_train_test(y_train, y_pred_train4, y_test, y_pred_test4)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5607    0.6648    0.6084       361
           2     0.7298    0.5714    0.6410       553
           3     0.5320    0.6242    0.5744       306

    accuracy                         0.6123      1220
   macro avg     0.6075    0.6201    0.6079      1220
weighted avg     0.6302    0.6123    0.6146      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2944    0.4013    0.3396       157
           2     0.4345    0.3080    0.3605       237
           3     0.2606    0.2846    0.2721       130

    accuracy                         0.3302       524
   macro avg     0.3298    0.3313    0.3241       524
weighted avg     0.3494    0.3302    0.3323       524



In [30]:
scores = get_metrics(y_train, y_pred_train4, y_test, y_pred_test4, "SVM", scores)
scores

<ipython-input-19-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.480328,0.480328,0.512694,0.482128,0.332061,0.332061,0.355855,0.333659
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.406489,0.406489,0.333390,0.337379
2,SVM,0.612295,0.612295,0.630170,0.614636,0.330153,0.330153,0.349380,0.332300


**KNN**

In [ ]:
knn = KNeighborsClassifier(n_neighbors=7)

In [ ]:
knn.fit(X=X_train, y=y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred_train5 = knn.predict(X_train)
y_pred_test5 = knn.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train5, y_test, y_pred_test5)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.5000    0.4986    0.4993       361
           2     0.5810    0.7523    0.6556       553
           3     0.5625    0.2647    0.3600       306

    accuracy                         0.5549      1220
   macro avg     0.5478    0.5052    0.5050      1220
weighted avg     0.5524    0.5549    0.5352      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.2975    0.2994    0.2984       157
           2     0.4823    0.6329    0.5474       237
           3     0.2909    0.1231    0.1730       130

    accuracy                         0.4065       524
   macro avg     0.3569    0.3518    0.3396       524
weighted avg     0.3794    0.4065    0.3799       524



In [ ]:
scores = get_metrics(y_train, y_pred_train5, y_test, y_pred_test5, "KNN", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.543443,0.543443,0.586143,0.484771,0.427481,0.427481,0.389838,0.362217
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.442748,0.442748,0.388654,0.382806
2,Adaboost,0.557377,0.557377,0.554195,0.534625,0.429389,0.429389,0.389787,0.392890
3,GBM,0.917213,0.917213,0.926445,0.917044,0.435115,0.435115,0.383880,0.384192
4,SVM,0.636066,0.636066,0.644885,0.622306,0.370229,0.370229,0.352319,0.357222
5,KNN,0.554918,0.554918,0.552394,0.535226,0.406489,0.406489,0.379445,0.379927


**GNB**

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()

In [ ]:
nb.fit(X=X_train, y=y_train)

GaussianNB()

In [ ]:
y_pred_train6 = nb.predict(X_train)
y_pred_test6 = nb.predict(X_test)

In [ ]:
classifcation_report_train_test(y_train, y_pred_train6, y_test, y_pred_test6)


               CLASSIFICATION REPORT FOR TRAIN DATA
            
              precision    recall  f1-score   support

           1     0.3918    0.1856    0.2519       361
           2     0.4901    0.5805    0.5315       553
           3     0.3071    0.3954    0.3457       306

    accuracy                         0.4172      1220
   macro avg     0.3963    0.3872    0.3764      1220
weighted avg     0.4151    0.4172    0.4021      1220


               CLASSIFICATION REPORT FOR TEST DATA
            
              precision    recall  f1-score   support

           1     0.3049    0.1592    0.2092       157
           2     0.4425    0.5359    0.4847       237
           3     0.2194    0.2615    0.2386       130

    accuracy                         0.3550       524
   macro avg     0.3222    0.3189    0.3108       524
weighted avg     0.3459    0.3550    0.3411       524



In [ ]:
scores = get_metrics(y_train, y_pred_train6, y_test, y_pred_test6, "GNB", scores)
scores

<ipython-input-21-3ef4ce65b0ea>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataframe       = dataframe.append(pd.Series([model_description,


,Model,Train_Accuracy,Train_Recall,Train_Precision,Train_F1_Score,Test_Accuracy,Test_Recall,Test_Precision,Test_F1_Score
0,DecisionTrees,0.543443,0.543443,0.586143,0.484771,0.427481,0.427481,0.389838,0.362217
1,RandomForest,1.000000,1.000000,1.000000,1.000000,0.442748,0.442748,0.388654,0.382806
2,Adaboost,0.557377,0.557377,0.554195,0.534625,0.429389,0.429389,0.389787,0.392890
3,GBM,0.917213,0.917213,0.926445,0.917044,0.435115,0.435115,0.383880,0.384192
4,SVM,0.636066,0.636066,0.644885,0.622306,0.370229,0.370229,0.352319,0.357222
5,KNN,0.554918,0.554918,0.552394,0.535226,0.406489,0.406489,0.379445,0.379927
6,GNB,0.417213,0.417213,0.415108,0.402142,0.354962,0.354962,0.345909,0.341115
